In [18]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

model = tf.keras.models.load_model('Face_Mask_Detection1.h5')


face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')


cap = cv2.VideoCapture(0)

while True:
    # Read a frame from the webcam
    ret, frame = cap.read()

    # Convert the frame to grayscale for face detection
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces using the Haar Cascade classifier
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5, minSize=(30, 30))

    
    for (x, y, w, h) in faces:
        # Extract the detected face region
        face_roi = frame[y:y + h, x:x + w]

        # Resize and preprocess the face region
        input_image_resized = cv2.resize(face_roi, (128, 128))
        input_image_scaled = input_image_resized / 255.0
        input_image_reshaped = np.reshape(input_image_scaled, [1, 128, 128, 3])

        # Make predictions for the face
        input_prediction = model.predict(input_image_reshaped)
        input_pred_label = np.argmax(input_prediction)
        confidence = np.max(input_prediction)

        # Display the prediction on the frame with bounding box if confidence is high
        if confidence > 0.5:  # Adjust the threshold as needed
            label = 'Without Mask' if input_pred_label == 1 else 'With Mask'
            
            color = (255, 255, 255) if input_pred_label == 1 else (255, 0, 0)
            
            cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2, cv2.LINE_AA)
            
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)

    # Display the frame with bounding boxes
    cv2.imshow('Real-Time Face Mask Detection', frame)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 23ms/step


1/1 [==============================] - 0s 63ms/step


1/1 [==============================] - 0s 22ms/step
